In [10]:
import pandas as pd
import numpy
import sqlite3
import pyTigerGraph as tg

In [11]:
# host = "http://medsales.34.106.148.222.nip.io:14240"
host = "http://localhost"
username = "tigergraph"
password = "Tigergraph"
graphName = "Northwind"
# mysecret = "iponcpkc1i73sjqld6i51utt9q2q8dkh"
mysecret = "ttlm3kccfsbm4n0bjp5f1272q3pvebap"
# First establish a basic connection using a secret.  Do *not* do this if you already have a token
# host = "http://34.106.148.222.nip.io"
# token="o4luvshk47rticuhqi0cdokiq739o8i9"
conn = tg.TigerGraphConnection(host=host, graphname=graphName, password=password)
token = conn.getToken(mysecret, setToken=True, lifetime=None)

# Next use the new token to establish a full access connection for use with GSQL

conn = tg.TigerGraphConnection(host=host, graphname=graphName, password=password, apiToken=token[0])


TigerGraphException: ("REST++ authentication is not enabled, can't generate token.", None)

In [ ]:
token[0]

In [ ]:
conn.getVertexTypes()

In [ ]:
conn.getVertexTypes()
conn.runInstalledQuery('floors')

In [3]:
orders = pd.read_csv("data/orders.csv")
order_details = pd.read_csv("data/order-details.csv")
customers = pd.read_csv("data/customers.csv")
reps = pd.read_csv("data/employees.csv")

In [4]:
order_details.dtypes


OrderID                  int64
ProductID                int64
UnitPrice              float64
Quantity                 int64
Discount               float64
LineTotalNoDiscount    float64
LineTotal              float64
dtype: object

In [5]:
order_details.describe().round()

,OrderID,ProductID,UnitPrice,Quantity,Discount,LineTotalNoDiscount,LineTotal
count,2155.0,2155.0,2155.0,2155.0,2155.0,2155.0,2155.0
mean,10659.0,41.0,26.0,24.0,0.0,629.0,587.0
std,241.0,22.0,30.0,19.0,0.0,1036.0,969.0
min,10248.0,1.0,2.0,1.0,0.0,5.0,5.0
25%,10451.0,22.0,12.0,10.0,0.0,154.0,147.0
50%,10657.0,41.0,18.0,20.0,0.0,360.0,338.0
75%,10862.0,60.0,32.0,30.0,0.0,722.0,656.0
max,11077.0,77.0,264.0,130.0,0.0,15810.0,15810.0


In [6]:
order_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2155 entries, 0 to 2154
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   OrderID              2155 non-null   int64  
 1   ProductID            2155 non-null   int64  
 2   UnitPrice            2155 non-null   float64
 3   Quantity             2155 non-null   int64  
 4   Discount             2155 non-null   float64
 5   LineTotalNoDiscount  2155 non-null   float64
 6   LineTotal            2155 non-null   float64
dtypes: float64(4), int64(3)
memory usage: 118.0 KB


In [7]:
order_details.head(20)

,OrderID,ProductID,UnitPrice,Quantity,Discount,LineTotalNoDiscount,LineTotal
0,10248,11,14.0,12,0.00,168.0,168.00
1,10248,42,9.8,10,0.00,98.0,98.00
2,10248,72,34.8,5,0.00,174.0,174.00
3,10249,14,18.6,9,0.00,167.4,167.40
4,10249,51,42.4,40,0.00,1696.0,1696.00
5,10250,41,7.7,10,0.00,77.0,77.00
6,10250,51,42.4,35,0.15,1484.0,1261.40
7,10250,65,16.8,15,0.15,252.0,214.20
8,10251,22,16.8,6,0.05,100.8,95.76
9,10251,57,15.6,15,0.05,234.0,222.30


In [8]:
%%bash
# rm -fv orders.db

In [9]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 830 entries, 0 to 829
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   OrderID         830 non-null    int64  
 1   CustomerID      830 non-null    object 
 2   EmployeeID      830 non-null    int64  
 3   OrderDate       830 non-null    object 
 4   RequiredDate    830 non-null    object 
 5   ShippedDate     809 non-null    object 
 6   ShipVia         830 non-null    int64  
 7   Freight         830 non-null    float64
 8   ShipName        830 non-null    object 
 9   ShipAddress     830 non-null    object 
 10  ShipCity        830 non-null    object 
 11  ShipRegion      323 non-null    object 
 12  ShipPostalCode  811 non-null    object 
 13  ShipCountry     830 non-null    object 
 14  OrderAmt        830 non-null    float64
 15  CommissionAmt   161 non-null    float64
dtypes: float64(3), int64(3), object(10)
memory usage: 103.9+ KB


In [ ]:
orders[orders['CommissionAmt'].notnull()]

In [ ]:
conn = sqlite3.connect('orders.db')
orders.to_sql('orders', conn, if_exists='replace')
order_details.to_sql('order_details', conn, if_exists='replace')
conn.close()

In [ ]:
reps = pd.read_csv("./data/employees.csv")

In [ ]:
reps['BirthDate']

# Reps and customers